In [31]:
import re
import pandas as pd
from io import StringIO
import nltk
import numpy as np

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

lemma = nltk.wordnet.WordNetLemmatizer()

ps = PorterStemmer()

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/matt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/matt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/matt/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [32]:
def extractDataFrame(fileName):
  with open(fileName) as file:
    lines = [re.sub(r'([^,])"(\s*[^\n])', r'\1/"\2', line) for line in file]
    df = pd.read_csv(StringIO(''.join(lines)), escapechar="/")
  return df

In [33]:
def splitLabels(labels):
  return [int(label) for label in labels.split(' ')]

In [34]:
def tokenise(caption):
    # Replace non-alphabetic characters with single whitespace
    caption = re.sub(r'[^a-zA-Z\s]', ' ', caption.lower())
    # Remove any whitespace that appears in sequence
    caption = re.sub(r"\s+", " ", caption)
    # Remove new leading and trailing whitespace
    caption = caption.strip()
    # Apply stemmer to each remaining word in sentence
    caption = lemma.lemmatize(caption)
    caption = ps.stem(caption)

    # Tokenize
    word_tokens = word_tokenize(caption)
    # Remove stop words
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    # Replace value with tokenised data
    return filtered_sentence

In [35]:
train = extractDataFrame('data/train.csv')

train['Tokens'] = train['Caption'].apply(tokenise)
train['Labels'] = train['Labels'].apply(splitLabels)

uniqueLabels = set()
for labels in train['Labels']:
  uniqueLabels.update(labels)

numUniqueLabels = len(uniqueLabels)
maxLabel = max(uniqueLabels)

print(uniqueLabels)
print('Unique labels:', numUniqueLabels)
print('Max label value:', maxLabel)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19}
Unique labels: 18
Max label value: 19


In [36]:
def labelMapping(uLabels):
  labelToIdMapping = {}
  idToLabelMapping = {}
  for idx, label in enumerate(uLabels):
    labelToIdMapping[label] = idx
    idToLabelMapping[idx] = label
  return labelToIdMapping, idToLabelMapping

In [37]:
labelToIdMapping, idToLabelMapping = labelMapping(uniqueLabels)

print(idToLabelMapping)

def mapLabelToId(labels):
  ids = []
  for label in labels:
    ids.append(labelToIdMapping[label])
  return ids

{0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 13, 12: 14, 13: 15, 14: 16, 15: 17, 16: 18, 17: 19}


In [38]:
def oneHotEncodeLabel(labels):
  oneHotClasses = np.zeros([numUniqueLabels, numUniqueLabels])
  for label in labels:
    # Onehot encode each label in the list
    oneHotClasses[label][label] = 1.
  return oneHotClasses.tolist()

In [39]:
train['LabelIds'] = train['Labels'].apply(mapLabelToId)
train['EncodedLabels'] = train['LabelIds'].apply(oneHotEncodeLabel)

print(train['Labels'][3])
print(train['LabelIds'][3])
print(train['EncodedLabels'][3])

[8, 3, 13]
[7, 2, 11]
[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [40]:
tokenLengths = []
for x in range(train.shape[0]):
    tokenLengths.append(len(train["Tokens"][x]))

maxTokenLength = max(tokenLengths)
print('Max token Length:', maxTokenLength)

Max token Length: 28


In [41]:
# Initialise vocabulary including padding and unknown word tokens
# Is end of sentence needed? '[EOS]'
vocabulary = set(['[PAD]', '[UNK]'])

for tokens in train["Tokens"]:
  vocabulary.update(tokens)

print(vocabulary)
print('Unique words:', len(vocabulary))

{'next', 'net', 'discussion', 'maneuvers', 'kayaks', 'hug', 'barhroom', 'headless', 'soccerball', 'desk', 'facil', 'carrots', 'staircase', 'clutching', 'tax', 'pirate', 'user', 'eare', 'marble', 'three', 'munch', 'jerseys', 'surfac', 'mall', 'onit', 'dogs', 'dora', 'swift', 'stroller', 'necklac', 'migrating', 'chatting', 'filming', 'flo', 'want', 'wrestler', 'range', 'televis', 'cemeteri', 'herds', 'native', 'rises', 'backlit', 'rugs', 'sheeep', 'skillfully', 'led', 'stage', 'begin', 'nun', 'safeway', 'diced', 'snapping', 'hat', 'honk', 'practic', 'steep', 'regulation', 'swimmers', 'chains', 'dives', 'triangle', 'hospital', 'tunnel', 'purchase', 'devil', 'rummaging', 'meter', 'aluminated', 'cardinal', 'obelisk', 'bare', 'pancak', 'baseline', 'trampolin', 'aaron', 'dong', 'lace', 'sel', 'pickup', 'merchandise', 'stole', 'miroor', 'company', 'clad', 'dwel', 'focu', 'dreamliner', 'goodbye', 'hotdog', 'pop', 'evening', 'greenhouse', 'mainten', 'main', 'bann', 'sponsored', 'quiet', 'portabl

In [42]:
dictionary = {}
idToDictionary = {}
for idx, word in enumerate(vocabulary):
  dictionary[word] = idx
  idToDictionary[idx] = word

# print(dictionary)

In [43]:
# Add dictionary IDs column to dataframe
def addDictionaryIds(tokens):
    ids = []
    for token in tokens:
      if token in dictionary:
        ids.append(dictionary[token])
      else:
        ids.append(dictionary['[UNK]'])
    return ids

In [44]:
train['DictionaryIds'] = train['Tokens'].apply(addDictionaryIds)

In [45]:
def addTokenPadding(tokens):
  padding = maxTokenLength - len(tokens)
  return tokens + padding*[dictionary['[PAD]']]

In [46]:
train['TokensWithPadding'] = train['DictionaryIds'].apply(addTokenPadding)
train.head()

,ImageID,Labels,Caption,Tokens,LabelIds,EncodedLabels,DictionaryIds,TokensWithPadding
0,0.jpg,[1],Woman in swim suit holding parasol on sunny day.,"[woman, swim, suit, holding, parasol, sunny, day]",[0],"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[5585, 3373, 2573, 5996, 1653, 4194, 3972]","[5585, 3373, 2573, 5996, 1653, 4194, 3972, 596..."
1,1.jpg,"[1, 19]",A couple of men riding horses on top of a gree...,"[couple, men, riding, horses, top, green, field]","[0, 17]","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[686, 3191, 3469, 1495, 1671, 3569, 3557]","[686, 3191, 3469, 1495, 1671, 3569, 3557, 5967..."
2,2.jpg,[1],They are brave for riding in the jungle on tho...,"[brave, riding, jungle, eleph]",[0],"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[4626, 3469, 5842, 7205]","[4626, 3469, 5842, 7205, 5967, 5967, 5967, 596..."
3,3.jpg,"[8, 3, 13]",a black and silver clock tower at an intersect...,"[black, silver, clock, tower, intersection, ne...","[7, 2, 11]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[3382, 2619, 855, 4078, 599, 2688, 4116]","[3382, 2619, 855, 4078, 599, 2688, 4116, 5967,..."
4,4.jpg,"[8, 3, 7]",A train coming to a stop on the tracks out side.,"[train, coming, stop, tracks, sid]","[7, 2, 6]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[3381, 3016, 2562, 5217, 1532]","[3381, 3016, 2562, 5217, 1532, 5967, 5967, 596..."


In [47]:
# Setup test data

# Get test CSV dataframe
test = extractDataFrame('data/test.csv')

# Tokenise
test['Tokens'] = test['Caption'].apply(tokenise)
# Apply dictionary IDs
test['DictionaryIds'] = test['Tokens'].apply(addDictionaryIds)
# Add padding
test['TokensWithPadding'] = test['DictionaryIds'].apply(addTokenPadding)

test.head()

,ImageID,Caption,Tokens,DictionaryIds,TokensWithPadding
0,30000.jpg,A little girl waring a krispy kreme hat holdin...,"[little, girl, waring, krispy, kreme, hat, hol...","[1162, 3897, 3950, 4033, 1057, 53, 5996, 4108,...","[1162, 3897, 3950, 4033, 1057, 53, 5996, 4108,..."
1,30001.jpg,A beautiful young woman holding an orange fris...,"[beautiful, young, woman, holding, orange, fri...","[3359, 3051, 5585, 5996, 3685, 3246]","[3359, 3051, 5585, 5996, 3685, 3246, 5967, 596..."
2,30002.jpg,A group of people sitting on couch next to a c...,"[group, people, sitting, couch, next, coffee]","[4067, 3198, 2469, 2364, 0, 1705]","[4067, 3198, 2469, 2364, 0, 1705, 5967, 5967, ..."
3,30003.jpg,A person on a snowboard rides on the hill.,"[person, snowboard, rides, hil]","[1736, 4729, 1782, 2242]","[1736, 4729, 1782, 2242, 5967, 5967, 5967, 596..."
4,30004.jpg,A man riding a skateboard with a helmet on in ...,"[man, riding, skateboard, helmet, con]","[5519, 3469, 5077, 2275, 7071]","[5519, 3469, 5077, 2275, 7071, 5967, 5967, 596..."


In [48]:
train.to_csv('./processed-data/train.csv')
test.to_csv('./processed-data/test.csv')

In [52]:
print(type(train['EncodedLabels'][0][0][0]))

<class 'float'>
